In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

In [3]:
# TEXTURE_SIZE = 400

_CHEEK_L = 0.35
_CHEEK_M = 0.5
_CHEEK_R = 0.65
_CHEEK_T = 0.3125
_CHEEK_B = 0.4125

_FACE_TIGHT_BOUNDS_L = 0.25
_FACE_TIGHT_BOUNDS_R = 0.75
_FACE_TIGHT_BOUNDS_T = 0.125
_FACE_TIGHT_BOUNDS_B = 0.625

In [4]:
from utils import uv

def get_fixed_process_uv(original_func):
    def fixed_process_uv(uv_coords, *args, **kwargs):
        return original_func(uv_coords.copy(), *args, **kwargs)
    return fixed_process_uv

uv.process_uv = get_fixed_process_uv(uv.process_uv)

In [5]:
import imageio
import cv2
import numpy as np
from tqdm.notebook import tqdm
import yaml

from FaceBoxes import FaceBoxes
from TDDFA import TDDFA
from utils.render import render, render_app
from utils.functions import cv_draw_landmark
from utils.depth import depth
from utils.pncc import pncc
from utils.uv import uv_tex, get_colors, process_uv, g_uv_coords
from utils.pose import viz_pose

from utils.tddfa_util import _to_ctype

In [6]:
def load_models():
    CONFIG = 'configs/mb1_120x120.yml'
    MODE = 'cpu'
    ONNX = True

    cfg = yaml.load(open(CONFIG), Loader=yaml.SafeLoader)

    # Init FaceBoxes and TDDFA, recommend using onnx flag
    if ONNX:
        import os
        os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
        os.environ['OMP_NUM_THREADS'] = '4'

        from FaceBoxes.FaceBoxes_ONNX import FaceBoxes_ONNX
        from TDDFA_ONNX import TDDFA_ONNX

        face_boxes = FaceBoxes_ONNX()
        tddfa = TDDFA_ONNX(**cfg)
    else:
        gpu_mode = MODE == 'gpu'
        tddfa = TDDFA(gpu_mode=gpu_mode, **cfg)
        face_boxes = FaceBoxes()
        
    return face_boxes, tddfa

In [8]:
def get_vertices_predictor(face_boxes, tddfa):
    dense_flag = True

    def predictor(frame_bgr):
        boxes = face_boxes(frame_bgr)
        boxes = [boxes[0]]
        param_lst, roi_box_lst = tddfa(frame_bgr, boxes)
        ver = tddfa.recon_vers(param_lst, roi_box_lst, dense_flag=dense_flag)[0]

        # refine
        param_lst, roi_box_lst = tddfa(frame_bgr, [ver], crop_policy='landmark')
        ver = tddfa.recon_vers(param_lst, roi_box_lst, dense_flag=dense_flag)[0]
        
        return ver

    return predictor

In [9]:
def bgr_to_rgb(image):
    if image.ndim == 3:
        image = image.copy()
        image[..., :3] = image[..., 2::-1]
    return image

In [10]:
import PIL
from IPython.display import display

def imshow(bgr):
    display(PIL.Image.fromarray(bgr_to_rgb(bgr)))

In [12]:
from matplotlib import pyplot as plt

def pltshow(bgr):
    plt.imshow(bgr_to_rgb(bgr))
    plt.gca().set_xticks([])
    plt.gca().set_yticks([])
    plt.show()

def show_channelwise(array):
    if isinstance(array, list) or isinstance(array, tuple):
        array = np.stack(array, axis=-1)
        
    n = array.shape[-1]
    for c in range(n):
        plt.subplot(1, n, c + 1)
        plt.imshow(bgr_to_rgb(array[..., c]))
        plt.gca().set_xticks([])
        plt.gca().set_yticks([])
    plt.gcf().set_size_inches(6 * n, 6)
    plt.show()

In [ ]:
def get_uv_coords(texture_size):
    assert texture_size > 0
    uvs = process_uv(g_uv_coords.copy(), uv_h=texture_size, uv_w=texture_size)
    return np.clip(np.round(uvs).astype(int), 0, texture_size - 1)


In [ ]:
def get_sparse_uv2ver(uv_coords, texture_size):
    assert texture_size > 0
    uv2ver_sparse = np.full((texture_size, texture_size, 3), fill_value=np.nan, dtype=np.float32)
    uv2ver_sparse[uv_coords[:, 1], uv_coords[:, 0]] = ver.T
    return uv2ver_sparse

In [13]:
from scipy import interpolate

# https://stackoverflow.com/questions/20516762/extrapolate-with-linearndinterpolator
class LinearNDInterpolatorExt(object):
    def __init__(self, points, values):
        self.funcinterp = interpolate.LinearNDInterpolator(points, values)
        self.funcnearest = interpolate.NearestNDInterpolator(points, values)
    
    def __call__(self, *args):
        z = self.funcinterp(*args)
        chk = np.isnan(z)
        if chk.any():
            return np.where(chk, self.funcnearest(*args), z)
        else:
            return z

In [14]:
# https://stackoverflow.com/questions/21690608/numpy-inpaint-nans-interpolate-and-extrapolate
def inpaint(image, mask, interpolator_cls):
    assert image.shape == mask.shape and image.ndim == 3
    image = np.where(mask, 0, image)
    
    nd = np.reshape(list(np.ndindex(image.shape[:2])), image.shape[:2] + (2, ))

    for c in range(image.shape[-1]):
        image_c = image[..., c]
        mask_c = mask[..., c]

        valid_mask = np.logical_not(mask_c) * (~np.isnan(image_c))
        coords = np.array(np.nonzero(valid_mask)).T
        values = image_c[valid_mask]

        it = interpolator_cls(coords, values)
        nd_masked = nd[np.where(mask_c)].reshape(-1, 2)
        image_c[np.where(mask_c)] = it(nd_masked)
    return image

In [22]:
def get_face_indicator(uv2ver, k=17):
    assert k >= 0
    face_indicator = (~np.isnan(uv2ver)).max(axis=-1).astype(np.uint8)
    
    if k > 0:
        face_indicator = cv2.erode(
            np.pad(face_indicator, pad_width=1, mode='constant', constant_values=0),
            np.ones((17, 17), np.uint8),
        )[1:-1, 1:-1]
    
    return face_indicator

In [15]:
from skimage.draw import line_aa

def draw_interpolated_line(src, dst, dst_w, pt1, pt2, blend_w_src=0.0):
    assert src.ndim == dst.ndim == dst_w.ndim == 3
    assert src.shape == dst.shape == dst_w.shape

    rr, cc, alpha = line_aa(*pt1[::-1], *pt2[::-1])

    tt = np.linspace(0.0, 1.0, num=len(alpha))
    
    v1 = src[pt1[1], pt1[0]]
    v2 = src[pt2[1], pt2[0]]
    interp = (v1[None,] * (1.0 - tt[..., None]) + v2[None,] * tt[..., None])
    
    assert 0.0 <= blend_w_src <= 1.0
    if 0.0 < blend_w_src:
        interp = interp * (1.0 - blend_w_src) + src[rr, cc] * blend_w_src
    
    dst[rr, cc] += interp * alpha[..., None]
    dst_w[rr, cc] += alpha[..., None]


In [23]:
from scipy.spatial import ConvexHull


def get_almost_convex_sparse_uv2ver(ver, uv2ver, face_indicator):

    def is_near_nose_tip(pt):
        return 0.45 <= (float(pt[0]) / uv2ver.shape[1]) <= 0.55 and \
               0.32 <= (float(pt[1]) / uv2ver.shape[0]) <= 0.42

    def is_higher_than_nose(pt):
        return (float(pt[1]) / uv2ver.shape[0]) <= 0.37

    def is_nasal_bridge(pt1, pt2):
        return (is_near_nose_tip(pt1) and is_higher_than_nose(pt2)) or \
               (is_near_nose_tip(pt2) and is_higher_than_nose(pt1))
    
    hull = ConvexHull(ver.T)

    convex_map_acc = np.zeros(uv2ver.shape, dtype=np.float32)
    convex_map_w = np.zeros(uv2ver.shape, dtype=np.float32)

    for simplex in hull.simplices:
        pts = [tuple(uv_coords[s, :2]) for s in simplex]
        if all(face_indicator[pt[1], pt[0]] == 0 for pt in pts): continue

        for st, en in [(pts[0], pts[1]), (pts[1], pts[2]), (pts[2], pts[0])]:
            blend_w_src = 0.75 if is_nasal_bridge(st, en) else 0.0
            draw_interpolated_line(uv2ver, convex_map_acc, convex_map_w, st, en, blend_w_src)

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        convex_map_sparse = convex_map_acc / convex_map_w
    assert np.isnan(convex_map_sparse).any()
    
    return convex_map_sparse

In [19]:
from scipy.ndimage import gaussian_filter

def blur_multichannel(array, *args, **kwargs):
    assert array.ndim == 3
    blurred = array.copy()
    for c in range(array.shape[-1]):
        blurred[..., c] = gaussian_filter(array[..., c], *args, **kwargs)
    return blurred

In [20]:
def get_blur_weights(shape):
    y_levels = np.mgrid[0:shape[0], 0:shape[1]][0].astype(np.float32) / shape[0]
    return np.clip(1.0 - (np.clip(y_levels, 0.37, 0.48) - 0.37) / 0.11, 0.2, 1.0)

In [21]:
from skimage.transform import warp
from random import uniform

class FacecoverTextureWarper:
    @staticmethod
    def _get_facecover2tex(facecover_size, texture_side):
        assert len(facecover_size) == 2 and all(v > 0 for v in facecover_size)
        assert texture_side > 0

        # X/Y coordinates
        ear_top_y = uniform(0.35, 0.425)
        facecover_mid_top_y = uniform(0.375, 0.420)
        facecover_top_y = uniform(0.26, 0.38)
        
        ear_bottom_y = uniform(0.615, 0.650)
        facecover_mid_bottom_y = uniform(0.66, 0.71)
        facecover_bottom_y = uniform(0.7, 0.8)
        
        facecover_near_top_y = facecover_top_y * 0.8 + facecover_bottom_y * 0.2
        facecover_near_bottom_y = facecover_bottom_y * 0.8 + facecover_top_y * 0.2
        
        facecover_half_width = uniform(0.18, 0.32)
        
        # Anchor points
        ear_lt = (0.05, ear_top_y)
        ear_lb = (0.10, ear_bottom_y)
        ear_rt = (0.95, ear_top_y)
        ear_rb = (0.90, ear_bottom_y)
        
        facecover_lmt = (0.5 - facecover_half_width, facecover_mid_top_y)
        facecover_lmb = (0.5 - facecover_half_width, facecover_mid_bottom_y)
        facecover_rmt = (0.5 + facecover_half_width, facecover_mid_top_y)
        facecover_rmb = (0.5 + facecover_half_width, facecover_mid_bottom_y)
        
        facecover_top = (0.5, facecover_top_y)
        facecover_near_top = (0.5, facecover_near_top_y)
        facecover_near_bottom = (0.5, facecover_near_bottom_y)
        facecover_bottom = (0.5, facecover_bottom_y)
        
        # Mapping from facecover-space to texture space
        
        facecover_left2tex = np.array([
            (0.0 , 0.0) + ear_lt,
            (0.25, 0.0) + facecover_lmt,
            (0.25, 1.0) + facecover_lmb,
            (0.0 , 1.0) + ear_lb,
        ])
        facecover_center2tex = np.array([
            (0.25, 0.0) + facecover_lmt,
            (0.5 , 0.0) + facecover_top,
            (0.5 , 0.2) + facecover_near_top,
            (0.75, 0.0) + facecover_rmt,
            (0.75, 1.0) + facecover_rmb,
            (0.5 , 0.8) + facecover_near_bottom,
            (0.5 , 1.0) + facecover_bottom,
            (0.25, 1.0) + facecover_lmb,
        ])
        facecover_right2tex = np.array([
            (1.0 , 0.0) + ear_rt,
            (0.75, 0.0) + facecover_rmt,
            (0.75, 1.0) + facecover_rmb,
            (1.0 , 1.0) + ear_rb,
        ])
        
        facecover2tex = [facecover_left2tex, facecover_center2tex, facecover_right2tex]
        
        for part2tex in facecover2tex:
            part2tex[:, :2] *= [facecover_size]
            part2tex[:, 2:] *= texture_side

        return facecover2tex
    
    @staticmethod
    def _to_float_image(image):
        if image.dtype == np.uint8:
            return image.astype(np.float32) / 255.0
        else:
            return image.copy()
        
    def __call__(self, facecover_uv_texture, texture_side):
        facecover_uv_texture = self._to_float_image(facecover_uv_texture)
            
        image = np.zeros((texture_side, texture_side, 4), dtype=np.float32)
        
        for part2tex in self._get_facecover2tex(facecover_uv_texture.shape[:2][::-1], texture_side):
            it = interpolate.CloughTocher2DInterpolator(part2tex[:, 2:], part2tex[:, :2])
            warped = warp(facecover_uv_texture, it, output_shape=(texture_side, texture_side, 4),
                          order=3, mode='constant', cval=0.0)
            # Alpha-blending
            image = warped * warped[..., -1:] + image * (1.0 - warped[..., -1:])

        return image

In [40]:
def get_mesh(uv2ver, face_indicator, num_rects_h, num_rects_w):
    assert num_rects_h > 0 and num_rects_w > 0

    face_vertice_to_idx = {}
    face_tri = []

    face_v_slices = np.linspace(0, num_rects_w - 1, num=num_rects_w)
    face_h_slices = np.linspace(0, num_rects_h - 1, num=num_rects_h)

    for face_vt, face_vb in zip(face_v_slices[:-1], face_v_slices[1:]):
        for face_vl, face_vr in zip(face_h_slices[:-1], face_h_slices[1:]):
            lt = (face_vl, face_vt)
            rt = (face_vr, face_vt)
            lb = (face_vl, face_vb)
            rb = (face_vr, face_vb)

            for v in [lt, rt, lb, rb]:
                v_idx = face_vertice_to_idx.get(v, len(face_vertice_to_idx))
                face_vertice_to_idx[v] = v_idx

            for vs in [(lt, rt, lb), (rb, lb, rt)]:
                face_tri.append([face_vertice_to_idx[v] for v in vs])

    verts_uv = np.zeros((len(face_vertice_to_idx), 2), dtype=int)
    for v, idx in face_vertice_to_idx.items():
        verts_uv[idx] = v

    face_ver = np.array([uv2ver[v, u] for u, v in verts_uv], dtype=np.float32).T
    ver_not_on_face = np.array([face_indicator[v, u] < 1 for u, v in verts_uv],
                               dtype=face_indicator.dtype)
    
    face_tri = np.array(face_tri, dtype=np.int32)
    tri_not_on_face = ver_not_on_face[face_tri].any(axis=-1)

    face_tri = face_tri[np.where(~tri_not_on_face)[0]]

    face_colors = np.array([[0, 0, 0] for u, v in verts_uv], dtype=np.float32)
    face_alphas = np.array([1.0 for u, v in verts_uv], dtype=np.float32)
    return dict(ver=face_ver, tri=face_tri, colors=face_colors, alphas=face_alphas, uv=verts_uv)

In [ ]:
def update_colors(mesh, facecover):
    mesh['colors'] = np.array([facecover[v, u][:3] for u, v in mesh['uv']], dtype=np.float32)
    mesh['alphas'] = np.array([facecover[v, u][-1] for u, v in mesh['uv']], dtype=np.float32)

In [ ]:
def get_visible_skin(uv2ver):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        visible_skin = (np.gradient(uv2ver[..., 0], axis=1) > 0.666).astype(np.float32)
    visible_skin[np.where(visible_skin < 1)] = np.nan
    return visible_skin

In [28]:
def is_both_sides_visible(visible_skin, threshold=0.1):
    left_sum = np.nansum(visible_skin[:, :visible_skin.shape[1] // 2])
    right_sum = np.nansum(visible_skin[:, visible_skin.shape[1] // 2:])
    return min(left_sum, right_sum) / max(left_sum, right_sum) > threshold

In [29]:
def calculate_ratio(left, right):
    return (-left / max(right, 1e-7) + 1) if left > right else (right / max(left, 1e-7) - 1)

In [33]:
def get_cheeks_lightness_ratio(texture, visible_skin):
    H, W = texture.shape[:2]
    lightness = cv2.cvtColor(texture, cv2.COLOR_BGR2HLS)[..., 1].astype(np.float32) / 255.0

    visible_lightness = np.where(np.isnan(visible_skin), np.nan, lightness)
    left_cheek = visible_lightness[int(H * _CHEEK_T):int(H * _CHEEK_B), int(W * _CHEEK_L):int(W * _CHEEK_M)]
    right_cheek = visible_lightness[int(H * _CHEEK_T):int(H * _CHEEK_B), int(W * _CHEEK_M):int(W * _CHEEK_R)]
    return calculate_ratio(np.nanmean(left_cheek), np.nanmean(right_cheek))

In [34]:
def find_light_x_position(target_ratio, verts_uv, ver, tri, texture, visible_skin, 
                          render_app, background_shape):
    if not is_both_sides_visible(visible_skin) or np.isnan(target_ratio):
        return 0
    
    original_light_pos = render_app.light_pos
    
    H, W = texture.shape[:2]

    left_cheek_ver_indices = np.array([
        i for i, (u, v, *_) in enumerate(verts_uv)
        if _CHEEK_L <= (float(u) / W) <= _CHEEK_M and _CHEEK_T <= (float(v) / H) <= _CHEEK_B
    ])
    right_cheek_ver_indices = np.array([
        i for i, (u, v, *_) in enumerate(verts_uv)
        if _CHEEK_M <= (float(u) / W) <= _CHEEK_R and _CHEEK_T <= (float(v) / H) <= _CHEEK_B
    ])
    
    tmp_img_placeholder = np.zeros(background_shape, dtype=np.uint8)

    light_ratios = {}
    
    for x_light in range(-10, 10, 1):
        render_app.light_pos = (x_light, 0, 5)
        light_texture = np.full(ver.T.shape, fill_value=1.0, dtype=np.float32)
        render_app(_to_ctype(ver.T), tri, tmp_img_placeholder, light_texture)

        left_cheek_mean = light_texture[left_cheek_ver_indices].mean()
        right_cheek_mean = light_texture[right_cheek_ver_indices].mean()

        if left_cheek_mean > right_cheek_mean:
            light_ratios[x_light] = -left_cheek_mean / max(right_cheek_mean, 1e-7) + 1
        else:
            light_ratios[x_light] = right_cheek_mean / max(left_cheek_mean, 1e-7) - 1

    render_app.light_pos = original_light_pos
    
    best_light_x = min(light_ratios.items(), key=lambda kv: abs(kv[1] - target_ratio))[0]
    # inc for positive, dec for negative, zero doesn't change
    best_light_x = best_light_x + best_light_x / max(abs(best_light_x), 1)
    return best_light_x

In [37]:
#pip install git+https://github.com/jrosebr1/color_transfer
from color_transfer import color_transfer

def facecover_color_transfer(facecover, texture):
    assert facecover.ndim == 3 and facecover.shape[-1] == 4
    H, W = texture.shape[:2]

    solid = facecover[..., :3].copy()
    alpha = facecover[..., -1]

    for c in range(3):
        solid[..., c][alpha < 0.1] = np.mean(solid[..., c][alpha > 0.1])

    with warnings.catch_warnings():
        warnings.filterwarnings('error')
        try:
            face_crop = texture[int(H * _FACE_TIGHT_BOUNDS_T):int(H * _FACE_TIGHT_BOUNDS_B),
                                int(W * _FACE_TIGHT_BOUNDS_L):int(W * _FACE_TIGHT_BOUNDS_R)]
            transferred = color_transfer(
                face_crop, (solid * 255.0).astype(np.uint8), clip=False, preserve_paper=False
            ).astype(np.float32) / 255.0
            transferred = cv2.addWeighted(transferred, 0.25, solid, 0.75, 0.0)
        except Warning:
            transferred = solid.copy()

    return np.concatenate([transferred, alpha[..., None]], axis=-1)

In [38]:
def get_lightning_params(target_ratio, light_x, facecover):
    assert facecover.ndim == 3 and facecover.shape[-1] == 4 and facecover.dtype == np.float32

    facecover_lightness = cv2.cvtColor(
        (facecover[..., :3] * facecover[..., -1:] * 255.0).astype(np.uint8),
        cv2.COLOR_BGR2HLS
    )[..., 1] / 255.0

    target_lightness = facecover_lightness.max()

    # heuristic: ambient + direct = ambient + ambient * (|ratio| + 1) = target_lightness
    ambient_w = target_lightness / (2.0 + abs(target_ratio))
    # second multiplier is another neuristic
    direct_w = (target_lightness - ambient_w) * np.power(abs(light_x) + 1, 0.125)
    return ambient_w, direct_w

In [39]:
def render_facecover(render_app, face_mesh, background, light_x, ambient_w, direct_w, 
                     return_intermediate=False):
    original_light_pos = render_app.light_pos
    original_light_intensity = (render_app.intensity_ambient,
                                render_app.intensity_directional)

    render_app.light_pos = (light_x, -2, 5)
    render_app.intensity_ambient = ambient_w
    render_app.intensity_directional = direct_w

    facecover_render = render_app(
        _to_ctype(face_mesh['ver'].T), face_mesh['tri'], background * 0, face_mesh['colors'].copy(),
    )
    
    render_app.intensity_ambient = 1.0
    render_app.intensity_directional = 0.0

    alpha_render = render_app(
        _to_ctype(face_mesh['ver'].T), face_mesh['tri'], background * 0,
        np.stack([face_mesh['alphas']] * 3, axis=-1)
    ).astype(np.float32) / 255.0
    
    prefinal_render = background * (1.0 - alpha_render) + facecover_render * alpha_render

    render_app.light_pos = original_light_pos
    render_app.intensity_ambient, render_app.intensity_directional = original_light_intensity
    
    # Light Wrap technique
    # https://www.imaging-resource.com/news/2016/02/11/create-natural-looking-composite-images-using-light-wrapping-technique
    alpha_rough = np.where(alpha_render[..., 0] > 0.25, 1.0, 0.0)
    alpha_rough_border = (np.maximum(*np.abs(np.gradient(alpha_rough))) > 0.0).astype(np.float32)
    
    alpha_light_wrap = gaussian_filter(alpha_rough_border, sigma=1.25, mode='nearest')[..., None] * 0.5
    background_blurred = blur_multichannel(background, sigma=2.5, mode='nearest')

    final_render = (
        prefinal_render * (1.0 - alpha_light_wrap) + background_blurred * alpha_light_wrap
    ).astype(np.uint8)
    
    if return_intermediate:
        return facecover_render, alpha_render, prefinal_render.astype(np.uint8), final_render
    return final_render